In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import torch
from datasets import load_dataset

c:\Users\playf\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_CHECKPOINT = "cointegrated/rubert-tiny2"
NUM_CLASSES = 30

In [5]:
dataset = load_dataset(
    "csv",
    data_files = "D:/Projects/Priorbank/Text-recognition/cleaned_data.csv"
)

ArrowInvalid: Expected to read 1954047348 metadata bytes, but only read 8635

In [39]:
def one_hotting(example):
    if isinstance(example['labels'], str):
        sep_labels = example['labels'].split(", ")
        one_hot = np.zeros(NUM_CLASSES)
        
        for lbl in sep_labels:
            one_hot[int(lbl)] = 1
        
        example['labels'] = one_hot.tolist()
    return example

mod_dataset = dataset.map(one_hotting)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

(3065, 30)

In [36]:
#input_tokens = tokenizer(texts, padding=True, 
#                        truncation=True, return_tensors="tf"
#                        )

In [26]:
model = BertForSequenceClassification.from_pretrained(MODEL_CHECKPOINT,
                                                        num_labels=NUM_CLASSES,
                                                        problem_type="multi_label_classification")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="D:/Projects/Priorbank/Text-recognition/training_args",
    num_train_epochs=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=mod_dataset["train"]
    )

In [33]:
#output = model(**input_tokens, labels=one_hot)

ResourceExhaustedError: Exception encountered when calling layer 'self' (type TFBertSelfAttention).

{{function_node __wrapped__RealDiv_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[3065,12,400,400] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:RealDiv] name: 

Call arguments received by layer 'self' (type TFBertSelfAttention):
  • hidden_states=tf.Tensor(shape=(3065, 400, 312), dtype=float32)
  • attention_mask=tf.Tensor(shape=(3065, 1, 1, 400), dtype=float32)
  • head_mask=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_value=None
  • output_attentions=False
  • training=False